In [1]:
import json
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Literal


class ChurnDriver(BaseModel):
    feature: str = Field(description="Name of the feature driving churn risk")
    impact: Literal["high", "medium", "low"] = Field(description="Impact level of this driver")
    reasoning: str = Field(description="Explanation of why this feature indicates churn risk")


class ChurnInsights(BaseModel):
    churn_risk_level: Literal["high", "medium", "low"] = Field(description="Overall churn risk assessment")
    top_drivers: List[ChurnDriver] = Field(description="Top 3-5 drivers of churn risk")
    narrative_summary: str = Field(description="2-3 sentence overview of customer situation")
    recommended_actions: List[str] = Field(description="Suggested intervention strategies")


def generate_churn_insights(passenger_features, model_prediction, model_confidence):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.3,
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    feature_descriptions = {
        "booking_freq_change": "Change in booking frequency over the last 90 days vs historical baseline.",
        "fare_downgrade_trend": "Recent shift from higher fare classes to cheaper ones, indicating reduced spend willingness.",
        "booking_abandon_rate": "Rate of searches started vs bookings completed, signaling intent to switch or indecision.",
        "recent_disruption_count": "Count of recent delays/cancellations in the last 3 trips that may cause dissatisfaction.",
        "recent_service_issues": "Baggage or service issues logged in the last 90 days.",
        "lounge_access_issues": "Instances of lounge denials/downgrades compared to customer's usual access.",
        "app_usage_drop": "Decline in app login frequency relative to baseline engagement.",
        "email_engagement_drop": "Decrease in email open/click rates over recent months.",
        "loyalty_inactivity_days": "Days since last loyalty program activity (earning, redemption, or tier progress).",
        "ancillary_spend_drop": "Reduced spending on ancillaries like meals, bags, and seat selection.",
        "upgrade_acceptance_drop": "Drop in acceptance of upgrade offers relative to past behavior.",
        "spend_per_trip_decline": "Decline in average customer spend per trip over the last 6 months.",
        "frustration_score_llm": "LLM-derived score reflecting negative sentiment or frustration in support interactions.",
        "complaint_frequency_change": "Increase in number of customer complaints or support tickets.",
        "churn_intent_signal_llm": "LLM-detected intent phrases that indicate thinking about switching airlines."
    }
    
    parser = PydanticOutputParser(pydantic_object=ChurnInsights)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an airline customer retention analyst specializing in churn prediction and customer behavior analysis."),
        ("user", """Analyze the following customer data and provide actionable churn insights.

Model Prediction: {prediction_label}
Model Confidence: {confidence}

Customer Features:
{features}

Feature Descriptions:
{descriptions}

Your task:
1. Identify the top 3-5 churn drivers based on the feature values
2. Explain the reasoning behind each driver
3. Provide a cohesive narrative about why this customer is at risk
4. Suggest intervention strategies

{format_instructions}""")
    ])
    
    chain = prompt | llm | parser
    
    result = chain.invoke({
        "prediction_label": "CHURN" if model_prediction == 1 else "NO CHURN",
        "confidence": f"{model_confidence:.2%}",
        "features": json.dumps(passenger_features, indent=2),
        "descriptions": json.dumps(feature_descriptions, indent=2),
        "format_instructions": parser.get_format_instructions()
    })
    
    return result.dict()


def analyze_passenger_batch(passengers_data):
    results = []
    
    for passenger in passengers_data:
        passenger_id = passenger.get('passenger_id', 'unknown')
        features = passenger['features']
        prediction = passenger['model_output']['prediction']
        confidence = passenger['model_output']['confidence']
        
        print(f"Analyzing passenger {passenger_id}...")
        
        insights = generate_churn_insights(features, prediction, confidence)
        
        results.append({
            'passenger_id': passenger_id,
            'model_prediction': prediction,
            'model_confidence': confidence,
            'insights': insights
        })
    
    return results


if __name__ == "__main__":
    sample_passenger = {
        'passenger_id': 'PAX_12345',
        'features': {
            'booking_freq_change': -0.45,
            'fare_downgrade_trend': 0.72,
            'booking_abandon_rate': 0.38,
            'recent_disruption_count': 2,
            'recent_service_issues': 3,
            'lounge_access_issues': 1,
            'app_usage_drop': -0.62,
            'email_engagement_drop': -0.55,
            'loyalty_inactivity_days': 87,
            'ancillary_spend_drop': -0.41,
            'upgrade_acceptance_drop': -0.33,
            'spend_per_trip_decline': -0.29,
            'frustration_score_llm': 0.78,
            'complaint_frequency_change': 2.5,
            'churn_intent_signal_llm': 0.65
        },
        'model_output': {
            'prediction': 1,
            'confidence': 0.87
        }
    }
    
    insights = generate_churn_insights(
        sample_passenger['features'],
        sample_passenger['model_output']['prediction'],
        sample_passenger['model_output']['confidence']
    )
    
    print("\nChurn Insights:")
    print(json.dumps(insights, indent=2))

ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/language_models/chat_models.py)